In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
open_eyes_dir = "/content/drive/MyDrive/ML_INNOVATIVE/train_new/Open"
closed_eyes_dir = "/content/drive/MyDrive/ML_INNOVATIVE/train_new/Closed"
yawn_dir = "/content/drive/MyDrive/ML_INNOVATIVE/train_new/yawn"
no_yawn_dir = "/content/drive/MyDrive/ML_INNOVATIVE/train_new/no_yawn"
posture_strong = "/content/drive/MyDrive/ML_INNOVATIVE/Posture/POSTURE/pos_strong"
posture_weak = "/content/drive/MyDrive/ML_INNOVATIVE/Posture/POSTURE/pos_weak"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
image_height, image_width = 100, 100
epochs = 20
batch_size = 32

In [ ]:
def load_and_preprocess_images(directory, label):
    images = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img = tf.keras.preprocessing.image.load_img(os.path.join(directory, filename), target_size=(image_height, image_width))
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            images.append(img_array)
            labels.append(label)

    return images, labels

In [ ]:
open_eyes_images, open_eyes_labels = load_and_preprocess_images(open_eyes_dir, 0)  # Label 0 for "open_eyes"
closed_eyes_images, closed_eyes_labels = load_and_preprocess_images(closed_eyes_dir, 1)  # Label 1 for "closed_eyes"
yawn_images, yawn_labels = load_and_preprocess_images(yawn_dir, 2)  # Label 2 for "yawn"
no_yawn_images, no_yawn_labels = load_and_preprocess_images(no_yawn_dir, 3)  # Label 3 for "no_yawn"
posture_strong_images, posture_strong_labels = load_and_preprocess_images(posture_strong, 4)  # Label 4 for "posture_strong"
posture_weak_images, posture_weak_labels = load_and_preprocess_images(posture_weak, 5)  # Label 5 for "posture_weak"

In [ ]:
X = np.concatenate((open_eyes_images, closed_eyes_images, yawn_images, no_yawn_images, posture_strong_images, posture_weak_images), axis=0)
y = np.concatenate((np.zeros(len(open_eyes_labels)), np.ones(len(closed_eyes_labels)), np.full(len(yawn_labels), 2),np.full(len(no_yawn_labels), 3), np.full(len(posture_strong_labels), 4), np.full(len(posture_weak_labels), 5)), axis=0)

In [ ]:
from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=42)

split_ratio = 0.8
split_index = int(len(X) * split_ratio)

X_train, y_train = X[:split_index], y[:split_index]
X_val, y_val = X[split_index:], y[split_index:]

In [ ]:
y_encoded = to_categorical(y, num_classes=6)

Model 1 - Convolutional Neural Network (CNN)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # Six classes: "open_eyes", "closed_eyes", "yawn", "no_yawn", "posture_strong", "posture_weak"
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_encoded[:split_index], epochs=epochs, validation_data=(X_val, y_encoded[split_index:]))

In [ ]:
test_loss, test_acc = model.evaluate(X_val, y_encoded[split_index:], verbose=2)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


y_pred = model.predict(X_val)


y_true = np.argmax(y_encoded[split_index:], axis=1)
y_pred = np.argmax(y_pred, axis=1)


cm = confusion_matrix(y_true, y_pred)


plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Open_Eyes', 'Closed_Eyes', 'Yawn', 'No_Yawn'], yticklabels=['Open_Eyes', 'Closed_Eyes', 'Yawn', 'No_Yawn'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# roc curve:
from sklearn.metrics import roc_curve, auc


fpr, tpr, _ = roc_curve(y_true, y_pred, pos_label=1)
roc_auc = auc(fpr, tpr)


plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
class_names = ['Open_Eyes', 'Closed_Eyes', 'Yawn', 'No_Yawn', 'Posture_Strong', 'Posture_Weak']
report = classification_report(y_true, y_pred, target_names=class_names)

print(report)

In [ ]:
# learning curve

plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Learning Curve')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# After training your model, save it to a file
model.save("/content/drive/MyDrive/ML_INNOVATIVE/model1_cnn.h5")

Prediction

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np


def check_drowsiness(model, image_path, image_height, image_width):
    # Load and preprocess the input image
    img = image.load_img(image_path, target_size=(image_height, image_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions using the model
    predictions = model.predict(img_array)

    # Define class labels
    class_labels = ['Open_Eyes', 'Closed_Eyes', 'Yawn', 'No_Yawn', 'Posture_Strong', 'Posture_Weak']

    # Determine drowsiness
    if predictions[0, 0] > 0.5 or predictions[0, 3] > 0.5:
        return "Not drowsy"
    else:
        return "Drowsy"

result = check_drowsiness(model, input_image_path, image_height, image_width)
print("Drowsiness Status:", result)


In [ ]:
# Load the trained model
model_path = "/content/drive/MyDrive/ML_INNOVATIVE/model1_cnn.h5"
model = tf.keras.models.load_model(model_path)

# Define the input image path
input_image_path = "/content/drive/MyDrive/ML_INNOVATIVE/Posture/POSTURE/pos_strong/2.jpg"
image_height = 100
image_width = 100

In [ ]:
def check_drowsiness(model, image_path, image_height, image_width):
    # Load and preprocess the input image
    img = image.load_img(image_path, target_size=(image_height, image_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions using the model
    predictions = model.predict(img_array)

    # Define class labels
    class_labels = ["open_eyes_no_yawn", "closed_eyes_no_yawn", "open_eyes_yawn", "closed_eyes_yawn", "posture_strong", "posture_weak"]

    # Thresholds for considering a class
    thresholds = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

    # Check if eyes are closed
    if predictions[0, 1] > thresholds[1]:
        return "Drowsy"

    # Check if eyes are open and there is a yawn
    elif predictions[0, 2] > thresholds[2]:
        return "Drowsy"

    # Check if eyes are open, no yawn, and weak posture
    elif predictions[0, 5] > thresholds[5]:
        return "Drowsy"

    # If none of the above conditions are met, consider as not drowsy
    return "Not drowsy"


In [ ]:
result = check_drowsiness(model, input_image_path, image_height, image_width)
print("Drowsiness Status:", result)

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from math import ceil

def predict_and_display_for_folder(model, folder_path, image_height, image_width, images_per_row=3):
    # Get a list of all files in the folder
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Calculate the number of rows needed
    num_rows = ceil(len(image_files) / images_per_row)

    # Iterate through each image and make predictions
    for row in range(num_rows):
        fig, axes = plt.subplots(1, images_per_row, figsize=(15, 5))

        for col in range(images_per_row):
            image_index = row * images_per_row + col
            if image_index < len(image_files):
                image_file = image_files[image_index]
                image_path = os.path.join(folder_path, image_file)

                # Load and preprocess the input image
                img = Image.open(image_path)
                img_array = np.array(img.resize((image_width, image_height))) / 255.0
                img_array = np.expand_dims(img_array, axis=0)

                # Make predictions using the model
                predictions = model.predict(img_array)

                # Thresholds for considering a class
                thresholds = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

                # Check if eyes are closed
                if predictions[0, 1] > thresholds[1]:
                    result = "Drowsy"

                # Check if eyes are open and there is a yawn
                elif predictions[0, 2] > thresholds[2]:
                    result = "Drowsy"

                # Check if eyes are open, no yawn, and weak posture
                elif predictions[0, 5] > thresholds[5]:
                    result = "Drowsy"

                # If none of the above conditions are met, consider as not drowsy
                else:
                    result = "Not drowsy"

                # Display the image and prediction
                axes[col].imshow(img)
                axes[col].set_title(f"{image_file}\nPrediction: {result}")
                axes[col].axis('off')

        plt.show()

In [ ]:
# Example usage:
folder_path = "/content/drive/MyDrive/ML_INNOVATIVE/test_new/JPG"
predict_and_display_for_folder(model, folder_path, image_height, image_width)

In [ ]:
import cv2
import matplotlib.pyplot as plt

def crop_eye(image_path):
    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        print("Failed to load the image.")
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Load the pre-trained haarcascade eye classifier
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Detect eyes in the image
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Crop the eyes from the image
    if len(eyes) > 0:
        (x, y, w, h) = eyes[0]
        cropped_eye = image[y:y+h, x:x+w]

        # Resize the cropped eye image to fit the input shape of the model
        resized_eye = cv2.resize(cropped_eye, (145, 145))

        # Preprocess the image for the model
        normalized_eye = resized_eye / 255.0
        input_data = normalized_eye.reshape(1, 145, 145, 3)

        # Display the cropped eye image
        plt.imshow(cv2.cvtColor(cropped_eye, cv2.COLOR_BGR2RGB))
        plt.title("Cropped Eye")
        plt.axis('off')
        plt.show()

        return cropped_eye  # Return the cropped eye image

    return None  # Return None if no eyes were detected



# Input image path
image_path = "/content/drive/MyDrive/ML_INNOVATIVE/train/IMG_1326.jpg"
cropped_eye = crop_eye(image_path)

cv2.imwrite("cropped_eye_open_h.jpg", cropped_eye)


Model 2 - Artificial Neural Network (ANN)

In [ ]:
num_classes = 6

In [ ]:
model_ann = Sequential([
    Flatten(input_shape=(image_height, image_width, 3)),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model_ann.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train_encoded = to_categorical(y_train, num_classes)
y_val_encoded = to_categorical(y_val, num_classes)

In [ ]:
epochs = 100
history = model_ann.fit(X_train, y_train_encoded, epochs=epochs, validation_data=(X_val, y_val_encoded))

In [ ]:
test_loss, test_acc = model_ann.evaluate(X_val, y_val_encoded, verbose=2)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

In [ ]:
# After training your model, save it to a file
model_ann.save("/content/drive/MyDrive/ML_INNOVATIVE/model1_ann.h5")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


Model 3 - EfficientNetB0

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

In [ ]:
model_eff = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model_eff.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
y_train_encoded = to_categorical(y_train, num_classes=4)
y_val_encoded = to_categorical(y_val, num_classes=4)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
test_losses = []
test_accuracies = []
fold_histories = []
epochs_eff = 10

In [ ]:
for train_indices, val_indices in kf.split(X):
    X_train, X_val = X[train_indices], X[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]

    y_train_encoded = to_categorical(y_train, num_classes=4)
    y_val_encoded = to_categorical(y_val, num_classes=4)

    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(4, activation='softmax')(x)

    model_eff = Model(inputs=base_model.input, outputs=predictions)

    model_eff.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    history = model_eff.fit(X_train, y_train_encoded, epochs=epochs_eff, validation_data=(X_val, y_val_encoded))

    fold_histories.append(history)

    test_loss, test_acc = model_eff.evaluate(X_val, y_val_encoded, verbose=0)
    print(f"Validation Loss: {test_loss}, Validation Accuracy: {test_acc}")

    test_losses.append(test_loss)
    test_accuracies.append(test_acc)

avg_test_loss = np.mean(test_losses)
avg_test_acc = np.mean(test_accuracies)
print(f"Average Test Loss: {avg_test_loss}, Average Test Accuracy: {avg_test_acc}")

In [ ]:
for i, history in enumerate(fold_histories):
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {i + 1} - Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {i + 1} - Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()